In [4]:
import pandas as pd
import json

# 지역 구분용 함수
def get_region(town):
    if town in ["구좌읍", "성산읍"]:
        return "동부지역"
    elif town in ["애월읍", "한림읍"]:
        return "서부지역"
    elif town in ["중문동", "남원읍", "표선면"]:
        return "남부지역"
    else:
        return "제주"

# 태그 자동 생성 함수
def generate_tags(row):
    tags = ["제주"]
    if pd.notna(row["분류명"]):
        tags += row["분류명"].split("/")  # ex: "관광안내소/매표소" → ["관광안내소", "매표소"]
    tags.append(get_region(row["법정동명"]))
    
    # 주소 기반 키워드 추출
    addr_fields = str(row.get("도로명주소명", "")) + str(row.get("리명", ""))
    if "올레" in addr_fields:
        tags.append("올레길")
    if "해안" in addr_fields or "바다" in addr_fields:
        tags.append("바다 전망")

    return list(set(tags))

# CSV 불러오기
df = pd.read_csv("../data/data.csv")

# JSON 리스트 생성
places = []
for _, row in df.iterrows():
    place = {
        "id": row["ID"],
        "name": row["POI명"],
        "category": "관광지",
        "theme": row["분류명"],
        "address": f"{row['시도명']} {row['시군구명']} {row['법정동명']} {row['리명']} {row['번지번호']}".strip(),
        "address_detail": f"{row.get('도로명주소명', '')} {row.get('건물번호', '')}".strip(),
        "lat": float(row["위치위도"]),
        "lng": float(row["위치경도"]),
        "tags": generate_tags(row),
        "description": "",
        "main_menu": "없음",
        "sub_menus": [],
        "opening_hours": "정보 없음",
        "closed": "정보 없음",
        "phone": ""
    }
    places.append(place)

# JSON 저장
with open("../data/관광지.json", "w", encoding="utf-8") as f:
    json.dump(places, f, ensure_ascii=False, indent=2)

In [7]:
import pandas as pd
import json
import glob

# 1. 음식점 CSV 병합
file_list = sorted(glob.glob("../data/음식점*.csv"))  # 음식점1.csv ~ 음식점5.csv
df_all = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)

# 2. 카페 필터링 기준 (포함된 단어 기반)
cafe_keywords = ["카페", "커피", "cafe", "CAFE"]

def is_cafe(name, description=""):
    combined = str(name) + " " + str(description)
    return any(keyword.lower() in combined.lower() for keyword in cafe_keywords)

df_all["is_cafe"] = df_all.apply(lambda row: is_cafe(row.get("name", ""), row.get("description", "")), axis=1)

# 3. 데이터 분리
df_cafe = df_all[df_all["is_cafe"] == True].copy()
df_food = df_all[df_all["is_cafe"] == False].copy()

# 4. JSON 통일 구조 변환 함수
def convert_df_to_json(df, category):
    places = []
    for idx, row in df.iterrows():
        place = {
            "id": f"{category[:2].upper()}_{idx:05d}",
            "name": row.get("name", f"{category}_{idx}"),
            "category": category,
            "address": row.get("address", ""),
            "lat": float(row.get("lat", 0)),
            "lng": float(row.get("lng", 0)),
            "tags": row.get("tags", "").split(",") if "tags" in row else [],
            "description": row.get("description", ""),
            "main_menu": row.get("main_menu", ""),
            "sub_menus": row.get("sub_menus", "").split(",") if "sub_menus" in row else [],
            "opening_hours": row.get("opening_hours", "정보 없음"),
            "closed": row.get("closed", "정보 없음"),
            "phone": row.get("phone", "")
        }
        places.append(place)
    return places

# 5. 각각 JSON으로 저장
with open("음식점_통합.json", "w", encoding="utf-8") as f:
    json.dump(convert_df_to_json(df_food, "음식점"), f, ensure_ascii=False, indent=2)

with open("카페_통합.json", "w", encoding="utf-8") as f:
    json.dump(convert_df_to_json(df_cafe, "카페"), f, ensure_ascii=False, indent=2)

In [10]:
import pandas as pd
import glob

# 1. 음식점1.csv ~ 음식점5.csv 파일 경로 패턴 지정 (현재 폴더 기준)
file_list = sorted(glob.glob("../data/음식점 [1-5].csv"))  # 예: 음식점1.csv, 음식점2.csv, ...

# 2. 각 파일을 DataFrame으로 읽어서 병합
df_all = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)

# 3. 병합된 데이터 저장
df_all.to_csv("음식점_통합.csv", index=False, encoding="utf-8-sig")

print("✅ 저장 완료: 음식점_통합.csv")

✅ 저장 완료: 음식점_통합.csv
